In [25]:
import pandas as pd

In [26]:
train=pd.read_csv('/content/drive/MyDrive/ulianov/train_exif.csv')
test=pd.read_csv('/content/drive/MyDrive/ulianov/test_exif.csv')

In [27]:
train['width']=train['x_max']-train['x_min']
train['height']=train['y_max']-train['y_min']

In [28]:
drops=['image_name','class']

train=train.drop(columns=drops)



In [29]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
pd.set_option('display.max_columns', 500)
from catboost import CatBoostRegressor

In [31]:
#print(train)
drops=['Make','Model','DateTime','HostComputer','ExifVersion','DateTimeOriginal',
       'DateTimeDigitized','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','SceneType',
       'LensMake','5']

train=pd.get_dummies(train,columns=['LensModel','1','3','12','16','23','Software'])
test=pd.get_dummies(test,columns=['LensModel','1','3','12','16','23','Software'])

train=train.drop(columns=drops)
test=test.drop(columns=drops)
print(train)

     distance       x_min        y_min        x_max        y_max      conf  \
0        4.88  219.604050   402.193756   425.297180   455.929901  0.842233   
1        1.54  614.961609  1326.627197  1302.240479  1479.057617  0.864385   
2        3.68  298.759644   461.289703   591.384766   529.101013  0.865617   
3        2.22  501.177734   748.439636   983.731384   856.448853  0.874792   
4        3.73  294.875153   471.319153   591.095154   539.254272  0.868262   
..        ...         ...          ...          ...          ...       ...   
525      3.00  306.027252   650.916504   604.249329   720.184814  0.865165   
526      3.46  269.867737   575.216919   522.212463   635.551270  0.865655   
527      3.91  249.696381   501.747467   478.498352   556.968689  0.857690   
528      5.20  198.687332   382.187531   368.827576   425.248566  0.837795   
529      5.54  181.400436   356.746887   344.161957   398.403748  0.829634   

     ResolutionUnit  ExifOffset  Orientation  XResolution  YRes

In [32]:
X=train.drop('distance',axis=1)
y=train['distance']
seed=42
cb=CatBoostRegressor(iterations=20000,random_state=seed,task_type='GPU',learning_rate=0.001)
cb.fit(X,y,verbose=1000)


test_vals=test.drop('image_name',axis=1)
preds=cb.predict(test_vals)

sub_d={
    'image_name':list(test[['image_name']].values),
    'distance':preds
}

sub=pd.DataFrame(sub_d)
sub['image_name']=list(map(lambda x:x[0],sub['image_name']))
print(sub)
sub.to_csv('/content/drive/MyDrive/ulianov/res.csv',index=False,sep=';')

0:	learn: 1.7557985	total: 33.9ms	remaining: 11m 17s
1000:	learn: 0.8866680	total: 13.1s	remaining: 4m 8s
2000:	learn: 0.5619732	total: 27.6s	remaining: 4m 8s
3000:	learn: 0.4440005	total: 36.9s	remaining: 3m 28s
4000:	learn: 0.3929854	total: 48s	remaining: 3m 12s
5000:	learn: 0.3650518	total: 57.3s	remaining: 2m 51s
6000:	learn: 0.3458650	total: 1m 6s	remaining: 2m 35s
7000:	learn: 0.3298487	total: 1m 15s	remaining: 2m 20s
8000:	learn: 0.3156296	total: 1m 25s	remaining: 2m 7s
9000:	learn: 0.3033050	total: 1m 34s	remaining: 1m 55s
10000:	learn: 0.2920368	total: 1m 44s	remaining: 1m 44s
11000:	learn: 0.2818786	total: 1m 54s	remaining: 1m 33s
12000:	learn: 0.2730023	total: 2m 3s	remaining: 1m 22s
13000:	learn: 0.2650082	total: 2m 13s	remaining: 1m 11s
14000:	learn: 0.2581311	total: 2m 22s	remaining: 1m
15000:	learn: 0.2519505	total: 2m 31s	remaining: 50.5s
16000:	learn: 0.2465127	total: 2m 41s	remaining: 40.2s
17000:	learn: 0.2414745	total: 2m 52s	remaining: 30.4s
18000:	learn: 0.2370155